In [ ]:
import sys
import os
import json
import pickle
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import CellModeller
from scipy.interpolate import interp1d
plt.rcParams['figure.figsize'] = 7, 7

#### Funcs

In [ ]:
# continuous model
def step(p1, p2, p3, gamma, mu, d, e, n, Dt):
    # Update protein concs
    nextp1 = p1 + ((d + e*(p3**n))/(1 + p3**n) - gamma*p1 - mu*p1) * Dt
    nextp2 = p2 + ((d + e*(p1**n))/(1 + p1**n) - gamma*p2 - mu*p2) * Dt
    nextp3 = p3 + ((d + e*(p2**n))/(1 + p2**n) - gamma*p3 - mu*p3) * Dt
    return nextp1, nextp2, nextp3

In [ ]:
# Gillespie's

#### Build lineage's graph from last pickle

In [ ]:
# using Deg03 to check correct values of 
path = '../Data/'
model = 'simpleGrowth10/'
files = os.listdir(path+model)
files.sort()
# not using module (think on removing it)
#files = files[1:]

In [ ]:
# get last pickle's lineage
last_pickle = pickle.load(open(path+model+files[-1], 'rb'))

In [ ]:
# constructing data
lin = last_pickle['lineage']
nodes = list(lin.keys())
edges = [(v,k) for k,v in lin.items()]

In [ ]:
# create directed graph
G = nx.DiGraph()
# add cell's ids as nodes
G.add_node(1)
G.add_nodes_from(nodes)
# add (parent_id, child_id) tuples as edges
G.add_edges_from(edges)

In [ ]:
# just testing
list(G.successors(1))

### Loop over pickles

#### Sim params

In [ ]:
# Continuous model params
Dt = 0.05
gamma = 0.3
d = 198.
e = 0
n = 2

# Sim params
pickeSteps = 1
#time between pickles 
p_time = Dt*pickeSteps #hours
store_folder = path+model[:-1]+'SIM/'

#### Initialize pickle 0 with desired initial conditions

In [ ]:
pickle_0 = pickle.load(open(path+model+'step-00000.pickle', 'rb'))
pickle_0['cellStates'][1].color = [0, 0, 5.0]
with open(store_folder+'step-00000.pickle', 'wb') as handle:
    pickle.dump(pickle_0, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#for idx in range(1, len(files)):
for idx in range(1, 1000):
    print("######################################################")
    print("idx: ", idx)
    # get pickles lineage
    # this pickle needs to have concs, so get it from pickle save past iteration
    data1 = pickle.load(open(store_folder+files[idx-1], 'rb')) 
    data2 = pickle.load(open(path+model+files[idx], 'rb'))

    # need to identify which cells divided from one pickle to the next
    # tells me which cells already divided when compared with previous pickle
    # will use: data['cellStates'].keys()

    # cells present in 2 but not in 1 (children)
    not_in_prev = list(set(data2['cellStates'].keys()) - set(data1['cellStates'].keys()))
    
    # cell division
    if len(not_in_prev) > 0:
        # cells in 1 and not in 2 (parents)
        not_in_curr = list(set(data1['cellStates'].keys()) - set(data2['cellStates'].keys()))

        # each cell in not_in_curr (parents)
        for par_cell in not_in_curr:
            succ = list(G.successors(par_cell))
            
            # SPLIT CONC from parent
            # 1) get conc from picke 1
            concs_1 = data1['cellStates'][par_cell].color
            # 2) split conc in 2
            new_conc = list(np.array(concs_1)/2)
            # 3) assign to each child
            data2['cellStates'][succ[0]].color = new_conc
            data2['cellStates'][succ[1]].color = new_conc
                    
            # cells remaining in not_in_prev obj
            new_left = list(set(not_in_prev) - set(succ))
            # update not_in_prev
            not_in_prev = new_left
            
            #print("not_in_prev: ", not_in_prev)
         
    # no cell division
    else:
        print("No cell division")
    

    # RUN CONTINUOUS OR GILLESPIE SIM
    # this is for cells that did't get divided
    #step(p1, p2, p3, gamma, mu, d, e, n, Dt)
    cells_both = list(set(data2['cellStates'].keys()) & set(data1['cellStates'].keys()))    
    
    # FOR EACH CELL
    for c in cells_both:
        # calculate concentrations
        concs = data1['cellStates'][c].color
        #print(f"conc for cell {c}: ", concs)
        p1, p2, p3 = concs[0], concs[1], concs[2]
        mu = data2['cellStates'][c].effGrowth
        nextp1, nextp2, nextp3 = step(p1, p2, p3, gamma, mu, d, e, n, Dt)
        
        # assign to next pickle
        data2['cellStates'][c].color[0] = nextp1
        data2['cellStates'][c].color[1] = nextp2
        data2['cellStates'][c].color[2] = nextp3
        
    # STORE PICKLE WITH NEW CONCS
    # needs to have all concs updated
    with open(store_folder+files[idx], 'wb') as handle:
        pickle.dump(data2, handle, protocol=pickle.HIGHEST_PROTOCOL)